# CS 425 fall 2021 Assignment 3


## Sequence alignment

### Part 1: Local alignment (70 pts)

Modify the following [code for global alignment](https://github.com/asabenhur/CS425/blob/main/notebooks/04_global_alignment.ipynb) to perform local sequence alignment of proteins using a substitution matrix.  In your code, use the [BLOSUM 62 substitution matrix](http://www.ncbi.nlm.nih.gov/Class/FieldGuide/BLOSUM62.txt).  Use your code to align the Aniridia protein from *H. sapiens* to the eyeless protein from *D. melanogaster*. The sequences of these proteins can be obtained from the Uniprot database: [Aniridia protein](http://www.uniprot.org/uniprot/P26367) and [eyeless protein](http://www.uniprot.org/uniprot/O18381).
Note that Uniprot has an option for saving an entry in Fasta format.

* What is the score of the optimal local alignment of the two proteins?  In the optimal local alignment produced by your code, what is the percent identity of the two sequences?  (The fraction of amino acids that are identical divided by the length of the alignment).

* Next, generate 100 random protein sequences with the same length and amino acid composition as the eyeless protein.  Align those sequences against the Aniridia protein, and compare the scores you obtained with those of the two PAX6 proteins, eyeless and Aniridia. What can you conclude from this about the relationship between the two PAX6 proteins?  



In [ ]:
import numpy as np
import random as rand
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
color = True
try :
    from termcolor import colored
except :
    color = False

# the three directions you can go in the traceback:
DIAG = 0 
UP = 1 
LEFT = 2

arrows = [u"\u2196", u"\u2191", u"\u2190"]

def needleman_wunsch_matrix(seq1, seq2, match=1, mismatch=-1, indel=-.5):
    """
    Fill the DP matrix according to the Needleman-Wunsch 
    algorithm for two sequences seq1 and seq2.
    match:  the match score
    mismatch:  the mismatch score
    indel:  the indel score
    
    Returns the matrix of scores and the matrix of pointers
    """
    n = len(seq1)
    m = len(seq2)
    s = np.zeros( (n+1, m+1) ) # DP matrix
    ptr = np.zeros( (n+1, m+1), dtype=int  ) # matrix of pointers

    ##### INITIALIZE SCORING MATRIX (base case) #####

    for i in range(1, n+1) :
        s[i,0] = indel * i
    for j in range(1, m+1):
        s[0,j] = indel * j
        
    ########## INITIALIZE TRACEBACK MATRIX ##########

    # Tag first row by LEFT, indicating initial '-'s
    ptr[0,1:] = LEFT
        
    # Tag first column by UP, indicating initial '-'s
    ptr[1:,0] = UP

    #####################################################
    MaxScore = -1000
    MaxScoreLocation = np.zeros( (n+1, m+1), dtype=int  )

    for i in range(1,n+1):
        for j in range(1,m+1): 
            # match
            if seq1[i-1] == seq2[j-1]:
                s[i,j] = s[i-1,j-1] + match

                if s[i,j] > MaxScore:
                  MaxScore = s[i,j]
                  row = i
                  col = j

                ptr[i,j] = DIAG
            # mismatch
            else :
                s[i,j] = s[i-1,j-1] + mismatch
                ptr[i,j] = DIAG
            # indel penalty
            if s[i-1,j] + indel > s[i,j] :
                s[i,j] = s[i-1,j] + indel
                ptr[i,j] = UP
            # indel penalty
            if s[i, j-1] + indel > s[i,j]:
                s[i,j] = s[i, j-1] + indel
                ptr[i,j] = LEFT

    
    return s, ptr, row, col, MaxScore

def needleman_wunsch_trace(seq1, seq2, s, ptr,row,col) :
    #### TRACE BEST PATH TO GET ALIGNMENT ####
    align1 = ""
    align2 = ""
    n, m = (row, col)
    startRow, startCol = (0,0)
    
    i = n
    j = m
    curr = ptr[i, j]

    while (i > startRow or j > startCol):        
        ptr[i,j] += 3
        if curr == DIAG :            
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1            
        elif curr == LEFT:
            align1 = '-' + align1
            align2 = seq2[j-1] + align2
            j -= 1            
        elif curr == UP:
            align1 = seq1[i-1] + align1
            align2 = '-' + align2
            i -= 1
           
        curr = ptr[i,j]
    
    return align1, align2


def show_ptr_matrix(ptr, seq1, seq2) :

    print('\n'+'~`'*25)
    print("Local Traceback")
    global color
    print(" " + " ".join(seq2))
    for i in range(len(ptr)) :
        if (i > 0) :
            print (seq1[i-1] + " ",end="")
        if (i == 0) :
            print("  ",end="")
        for j in range(len(ptr[i])) :
            if color and ptr[i,j] >= 3 :
                print(" " + colored(arrows[ptr[i,j]-3], 'green' ),
                      end="")
            else :
                if ptr[i,j] >=3 :
                    ptr[i,j] -=3
                print(" " + arrows[ptr[i,j]],end="")
        print() 

def show_dp_matrix(s, seq1, seq2) :

    print('\n'+'~`'*25)
    print("Global DP matrix")
    print("         " + "    ".join(seq2))
    for i in range(len(s)) :
        if (i > 0) :
            print(seq1[i-1] + " ",end="")
        if (i == 0) :
            print("  ",end="")
        for j in range(len(s[i])) :
            print(" " + "% 2.1f" % s[i,j],end="")
        print()

In [ ]:

def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, indel=-.5, verbose=True) :
    """
    computes an optimal global alignment of two sequences using 
    the Needleman-Wunsch algorithm
    returns the alignment and its score
    """
    s,ptr,row,col,maxscore = needleman_wunsch_matrix(seq1, seq2, match, mismatch, indel)
    alignment = needleman_wunsch_trace(seq1, seq2, s, ptr,row,col)

    if verbose :
        # show_dp_matrix(s, seq1, seq2)
        # show_ptr_matrix(ptr, seq1, seq2)
        print('\n'+'~`'*25)
        print("Local Alignment Score: %f\n" % (maxscore))
        print("Alignment:")
        print(alignment[0])
        print(alignment[1])
    

    return alignment, s[len(seq1), len(seq2)]

In [ ]:
def read_fasta(file):
  with open(file,"r") as f:
    #read first line of description
    description = f.readline().rstrip()

    #read proteins
    proteins = ""
    line = f.readline().rstrip()
    while line != '':
      proteins += line
      line = f.readline().rstrip()
  return proteins


def proteinSequence():
  proteins = 'ARNDCQEGHILKMFPSTWYV'

  store = ''.join(rand.choice(proteins) for i in range(857))

  return store

if __name__ == '__main__':
  aniridia = read_fasta('P26367.fasta')
  eyeless = read_fasta('O18381.fasta')

  print('Aniridia and Eyeless local alignment')
  needleman_wunsch('-' + aniridia, '-' + eyeless, )
  print()
  print()
  print()

  for i in range(100):
    print('String Number:', i+1)
    needleman_wunsch(aniridia, proteinSequence())
    print()
    print()
    print()

# Analysis
**Data Collection**
>To complete the task given to me this time, I needed to use two FASTA files that were downloaded from uniprot.org to download the protein sequences of Aniridia and Eyeless proteins. The size of Aniridia is 422 characters long and the size of the eyeless sequence is 857 characters long. While it is not data, I was also given code provided by Dr.Asa Ben-Hur which creates a global alignmnet of the 2 sequences.

**Data Quality Check And Cleaning**
>This FASTA file came in just like most FASTA files with a description about the sequence followed by the actual data sequence needed to complete my task. I used the same read_fasta() function i've been using since it always works great with the given FASTA files. FIGURE 1

```
#FIGURE 1
def read_fasta(file):
  with open(file,"r") as f:
    #read first line of description
    description = f.readline().rstrip()

    #read proteins
    proteins = ""
    line = f.readline().rstrip()
    while line != '':
      proteins += line
      line = f.readline().rstrip()
  return proteins
```
>Since I had no use for the description I stripped the first line but never returned it or had a use for it. The rest of the code just reads each line from the file and returns one singular string of the protein sequence.

**Data Processing**
>Processing the data was really diffrent this time around compared to the previous times. This is because I was actaully given code that completed a golbal alignment. What I did for the first hour or so was understanding what the code was doing then changing parts of the code to see how it will work with different factors. For example, I tested the code with only part of a sequence to see how the program would handle it. Also change some variables to see what the output will be. I also needed to check what type of inputs or variables changes will break the code and return an error of any type. Once I felt conmftroable with the code given to me I was ready to commit the changes needed to output a local alignment compared to a global alignment. The first step in the program is to extract the data from the FASA file in which I explain up above on what I did. Next I needed to call the function needleman_wunsch() which all it really does is call other functions to create the matrix and traceback. Then all it does next it print them out. As far as other functions go this is a pretty basic one that does mostly prints. FIGURE 2

```
# FIGURE 2
def needleman_wunsch(seq1, seq2, match=5, mismatch=-1, indel=-.5, verbose=True):

    s,ptr,row,col,maxscore = needleman_wunsch_matrix(seq1, seq2, match,mismatch, indel)

    alignment = needleman_wunsch_trace(seq1, seq2, s, ptr,row,col)

    if verbose :
        # show_dp_matrix(s, seq1, seq2)
        # show_ptr_matrix(ptr, seq1, seq2)
        print('\n'+'~`'*25)
        print("Local Alignment Score: %f\n" % (maxscore))
        print("Alignment:")
        print(alignment[0])
        print(alignment[1])
    
    return alignment, s[len(seq1), len(seq2)]
```
>The next function that gets called is needleman_wunsch_matrix() and this functions job is to create the actual scoring matrix. It stars off with a NumPy array filled with zeros that will change once the code is running. It checks both strings at a certain index. Then it gives a score depending on if it is a match, mismatch or indel. I did need to edit the code to find the best local allignment.FIGURE 3


```
# FIGURE 3
 if s[i,j] > MaxScore:
                  MaxScore = s[i,j]
                  row = i
                  col = j
```
>As you can see it was a small addition that helps me find the local alignmnet.
I have a variable called MaxScore that is set to a low number and whenever we have a new high score then I set MaxScore to that number. I also keep track of the row and colloumn it was located in so I can pass that on to the next function so it knows to start the traceback at that point. The rest of the code was left untouched sinced I still needed the entire matrix to be filled with the scores. The next part of code the start running is called needleman_wunsch_trace(). In here I just needed to add the row and coloumn variables into the code so it knows that first of all I want the highest score of the local alignmnet and also that I want the traceback to start going back from that high score local alignment. At first the code was originally taking in the length of both sequences so it could start the tracback all the way at the end for the global alignment. So, all I neded to do was change those 2 variables and set them to row and colounmn variables that was returned from the prevoius function. FIGURE 4

```
# FIGURE 4
  n, m = (row, col)
```
> Now the traceback knows exactly where to start to get the local alignment. The rest of the code is left untouched since I still want to get every single traceback but only return green arrows for the local alignment.
We return back to the needleman_wunsch() function and all it has left to do is to print out the matrix, traceback, and the local alignment sequence. As of right now I have the commented out the matrix and traceback since the length of both strings are over 400 characters long. So when the matrix is printed out it can be really hard to see the scores and the tracback.

**Data Analysis and Questions**
>All the code is ready to be ran fully so I made sure that I set the scores of Match, Mismatch and indel are set accordingly. When I ran the code with the eyeless and anirdia sequences I got back the local alignmnet score of 98.5 and the alignmnet sequence being. 
```
-MQN--------------------------------------S--------------HSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRPRAIGGSKPRVATPEVVSKIAQYKRECPSIFAWEIRDRLLSEGVCTNDNIPSVSSINRVLRNL-A-SEKQQ--------------------------------------------M------------GA----DG-----MYDKLRMLN-------------------GQ------T---------G-----------SWGTR--PG-WYPGT--------------SV---------------P---------G-Q---P--T--------QDGCQQQE-G-G-GENTN---S-ISSNGEDSDEAQMRLQLKRKLQRNRTSFTQEQIEALEKEFERTHYPDVFARERLAAKIDLPEARIQVWFSNRRAKWRREEKLRNQRRQASNTP-SHIPIS--SSFSTS
-MRNLPCLGTAGGSGLGGIAGKPSPTMEAVEASTASHRHSTSSYFATTYYHLTDDECHSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETGSIRPRAIGGSKPRVATAEVVSKISQYKRECPSIFAWEIRDRLLQENVCTNDNIPSVSSINRVLRNLAAQKE-QQSTGSGSSSTSAGNSISAKVSVSIGGNVSNVASGSRGTLSSSTDLMQTATPLNSSESGGASNSGEGSEQEAIYEKLRLLNTQHAAGPGPLEPARAAPLVGQSPNHLGTRSSHPQLVHGNHQALQQHQQQSWPPRHYSGSWYP-TSLSEIPISSAPNIASVTAYASGPSLAHSLSPPNDIESLASIGHQRNCPVATEDIHLKKELDGHQSDETGSGEGENSNGGASNI-GNTED-D--QARLILKRKLQRNRTSFTNDQIDSLEKEFERTHYPDVFARERLAGKIGLPEARIQVWFSNRRAKWRREEKLRNQRR----TPNS-TGASATSS-STS
```
While it did return a lot of indels we can look closely and see there are 2 subsequences in which both strings share identical proteins. The percent identity of the 2 sequences is 47.9%. Next I randomly created 100 strings with diffrent sequences of protein and used that to see what kind of alignments I could get when compared with the aniridia sequence. Using random sequences showed me that the eyeless and aniridia sequences are very similar since the highest score I got back was a 1 and the lowest scoe being a -6.5. Below I will show you what the local alignment sequence was when running the code.
```
Local Alignment Score: -6.500000
Alignment:
-MQNSHSGV
TKYAWPTYV
Local Alignment Score: 0.000000
Alignment:
MQ
EQ
```
>This tells me that the eyeless and aniridia sequences share strong similarities because even when creating 100 random strings of amino acids, it still struglled getting a high local alignment score.

### Part 2:  Global alignment variant (15 pts)

Devise a dynamic programming algorithm that finds an optimal global alignment between two sequences such that the alignment contains at most k blocks of consecutive indels.


>In the function needleman_wunsch(), it returns the 2 alignment sequences. What I would have it do then is go through each sequence and find the biggest "stretch" of indels. I would iterate thorugh the string and as soon as I find an indel i'd keept track of its position and then count how many more indels follow that position and keep track of that number. Once it goes thorugh the entire string I can see where the string had the most amount of indels in a row. Since I would be keeping track of it's position I would be able to splice the string of certain positions to return the desire sequence alignment.

### Part 3:  Global alignment without end gap penalties (15 pts)

Propose a modification of the Needleman Wunsch algorithm such that there is no penalty for indels at the end of the alignment.  There should be penalties for gaps at the beginning of the alignment.

>The modification I would make would be quite a simple one to make. Since I keep track of where the traceback begins in 2 variables called row and col. I would just run the function needleman_wunsch_matrix() again except this time ill pass it the row and col so it knows that after this index in the NumPy array, there should be no penalties after it.


### Submission

Submit your assignment as a Jupyter notebook via Canvas.  

### Grading 

Here is what the grading sheet will look like for this assignment. 

```
Grading sheet for assignment 3

Part 1
- Implementation of SW with a substitution matrix (50 pts)
- Results and analysis (20 pts)
Part 2 (15 pts)
Part 3 (15 pts)

```
